# Project III

In [1]:
import json
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

from pymongo import MongoClient
import pandas as pd
import time 



### Connecting to MongoDB

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["ironhack"]

In [4]:
c = db.get_collection("companies")

In [5]:
c.distinct("offices")


[{'description': None,
  'address1': None,
  'address2': None,
  'zip_code': None,
  'city': None,
  'state_code': None,
  'country_code': 'CAN',
  'latitude': 56.130366,
  'longitude': -106.346771},
 {'description': None,
  'address1': None,
  'address2': None,
  'zip_code': None,
  'city': None,
  'state_code': None,
  'country_code': 'GBR',
  'latitude': 55.378051,
  'longitude': -3.435973},
 {'description': None,
  'address1': None,
  'address2': None,
  'zip_code': None,
  'city': None,
  'state_code': None,
  'country_code': 'ISR',
  'latitude': 31.046051,
  'longitude': 34.851612},
 {'description': None,
  'address1': None,
  'address2': None,
  'zip_code': None,
  'city': None,
  'state_code': None,
  'country_code': 'JPN',
  'latitude': 36.204824,
  'longitude': 138.252924},
 {'description': None,
  'address1': None,
  'address2': None,
  'zip_code': None,
  'city': None,
  'state_code': None,
  'country_code': 'NLD',
  'latitude': 52.132633,
  'longitude': 5.291266},
 {'descr

In [6]:
condition_1 =  {"total_money_raised": {"$regex": "M$|B$"}} 
condition_2 = {"tag_list":{"$regex": "design"}}  
condition_3 = {"offices.country_code": {"$exists": True}}   


query = {"$and": [condition_1,condition_2, condition_3]}
         

projection = {"offices.city":1,"offices.country_code":1, "offices.state_code":1,"offices.latitude":1,"offices.longitude":1,"name":1, "tag_list":1, "total_money_raised":1, "_id":0}

df=list(c.find(query, projection))

After pushing, view your own notebook on [nbviewer](https://nbviewer.org/)

In [7]:
world_offices_data = pd.DataFrame(df)


In [8]:
x= world_offices_data.iloc[0]["offices"]
x[0]['latitude']


37.764726

In [9]:




# Extract data for the first office
world_offices_data["lat"] = world_offices_data["offices"].apply(lambda x: x[0]['latitude'] if len(x) >= 1 else None)
world_offices_data["lon"] = world_offices_data["offices"].apply(lambda x: x[0]['longitude'] if len(x) >= 1 else None)
world_offices_data["state_code"] = world_offices_data["offices"].apply(lambda x: x[0]['state_code'] if len(x) >= 1 else None)
world_offices_data["country_code"] = world_offices_data["offices"].apply(lambda x: x[0]['country_code'] if len(x) >= 1 else None)
world_offices_data["city"] = world_offices_data["offices"].apply(lambda x: x[0]['city'] if len(x) >= 1 else None)

# Split rows with two offices into two separate rows
world_offices_data = world_offices_data.explode("offices")

# Extract data for the second office (new rows)
world_offices_data["lat"] = world_offices_data["offices"].apply(lambda x: x['latitude'] if len(x) >= 2 else None)
world_offices_data["lon"] = world_offices_data["offices"].apply(lambda x: x['longitude'] if len(x) >= 2 else None)
world_offices_data["state_code"] = world_offices_data["offices"].apply(lambda x: x['state_code'] if len(x) >= 2 else None)
world_offices_data["country_code"] = world_offices_data["offices"].apply(lambda x: x['country_code'] if len(x) >= 2 else None)
world_offices_data["city"] = world_offices_data["offices"].apply(lambda x: x['city'] if len(x) >= 2 else None)



world_offices_data.dropna(axis=0, inplace=True)
world_offices_data=world_offices_data.drop("offices", axis=1)
world_offices_data = world_offices_data.reset_index()
world_offices_data





,index,name,tag_list,total_money_raised,lat,lon,state_code,country_code,city
0,0,Digg,"community, social, news, bookmark, digg, techn...",$45M,37.764726,-122.394523,CA,USA,San Francisco
1,1,GENWI,"mcms, mobile-content-management-system, live-a...",$7.1M,33.817100,-111.903500,CA,USA,Los Altos
2,4,Gilt Groupe,"clothing, fashion, designer, apparel, luxury, ...",$236M,40.747270,-73.980064,NY,USA,New York
3,5,Sitemasher,"website-design-and-management-platform, saas",C$5M,25.705057,-80.271378,FL,USA,Coral Gables
4,6,99designs,"graphic-design, crowdsourcing, online-marketpl...",$35M,37.795531,-122.400598,CA,USA,San Francisco
5,8,Shopflick,"video, shopping, videoshopping, home-shopping,...",$8M,34.044939,-118.419067,CA,USA,Los Angeles
6,9,deviantART,"art, digital-art, photography, design, graphic...",$13.5M,34.101578,-118.333340,CA,USA,Hollywood
7,10,Minted,"crowd-sourcing, design, art, stationery, ecomm...",$52.7M,37.797435,-122.403175,CA,USA,San Francisco
8,14,Azuro,"chip-design, semiconductor, eda",$10M,37.406217,-121.976473,CA,USA,Santa Clara
9,15,NextG Networks,"wireless, network, network-architecture, distr...",$410M,37.396808,-121.911363,CA,USA,Milpitas


In [10]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

In [11]:

usa_lat = 37.7576171
usa_lon = -122.5776844
usa_map = Map(location = [usa_lat, usa_lon], zoom_start = 15)

for index, row in world_offices_data.iterrows():
    all_together = {"location": [row["lat"], row["lon"]]}
    icon = Icon (
                icon = "building",
                prefix = "fa",
                color = "ce40c2",
                icon_color = "white")        
       
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(usa_map)

usa_map

/var/folders/tb/spjhzdz13378nkrzqyjljkjm0000gn/T/ipykernel_2705/3011067261.py:7: UserWarning: color argument of Icon should be one of: {'beige', 'darkpurple', 'green', 'purple', 'cadetblue', 'red', 'white', 'lightblue', 'lightgreen', 'darkgreen', 'orange', 'darkred', 'lightgray', 'blue', 'black', 'lightred', 'pink', 'darkblue', 'gray'}.
  icon = Icon (


### As we can see most of the startups that meet the following requirements are in the west coast:

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.

- Developers like to be near successful tech startups that have raised at least 1 Million dollars.

### We will focus on Mountain View and San Francisco

## Let's start with the API

In [12]:
from getpass import getpass
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()
token = os.getenv("token")

def query_from_frousquare (query, lat, lon, radius, limit):
    
    
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    response = requests.get(url, headers=headers).json()
    return response

## Let's start with schools

#### San Francisco and Mountain View


In [14]:
import requests
response=query_from_frousquare ("elementary school", 37.764726, -122.394523, 500, 5)
response

{'results': [{'fsq_id': '4ada1e70f964a520a01e21e3',
   'categories': [{'id': 12058,
     'name': 'Elementary School',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/school_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 441,
   'geocodes': {'main': {'latitude': 37.760587, 'longitude': -122.395863},
    'roof': {'latitude': 37.760587, 'longitude': -122.395863}},
   'link': '/v3/places/4ada1e70f964a520a01e21e3',
   'location': {'address': '465 Missouri St',
    'census_block': '060750227022008',
    'country': 'US',
    'cross_street': 'at 20th St',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '465 Missouri St (at 20th St), San Francisco, CA 94107',
    'locality': 'San Francisco',
    'postcode': '94107',
    'region': 'CA'},
   'name': 'Daniel Webster Elementary SCHL',
   'related_places': {},
   'timezone': 'America/Los_Angeles'},
  {'fsq_id': '4af2f8fdf964a52079e921e3',
   'categories': [{'id': 12058,
     'name': 'Elem

In [15]:
response["results"][0]["categories"][0]["name"]

'Elementary School'

In [16]:
response["results"][0]["distance"]

441

In [17]:
response["results"][0]["geocodes"]["main"]["latitude"]

37.760587

In [18]:
response["results"][0]["location"]["formatted_address"]

'465 Missouri St (at 20th St), San Francisco, CA 94107'

In [19]:
response["results"][0]["geocodes"]["main"]["longitude"]

-122.395863

In [20]:
def factor_from_foursquare (one_element):
    name=response["results"][0]["categories"][0]["name"]
    distance=response["results"][0]["distance"]
    address=response["results"][0]["location"]["formatted_address"]
    lat=response["results"][0]["geocodes"]["main"]["latitude"]
    lon=response["results"][0]["geocodes"]["main"]["longitude"]
    
    
    dict_ = {"name": name, "address":address, "distance": distance, "lat": lat, "lon": lon}
    
    return dict_



In [21]:
new_list = []
for i in response["results"]:
    new_list.append(factor_from_foursquare (i))

df = pd.DataFrame(new_list)
df

,name,address,distance,lat,lon
0,Elementary School,"465 Missouri St (at 20th St), San Francisco, C...",441,37.760587,-122.395863
1,Elementary School,"465 Missouri St (at 20th St), San Francisco, C...",441,37.760587,-122.395863
